In [3]:

import os, sys

import numpy as np
import pandas as pd
import xarray as xr
from glob import glob # useful package for opening files


# This just changes the display of xarray datasets so they don't automatically
# have expanded attributes (attrs)
xr.set_options(display_expand_attrs=False)

# Can change this to False to stop xarray displaying all the data
xr.set_options(display_expand_data=True)

In [4]:
# Making sure the notebook can access the files in your google drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ROOT_DIR = '/content/drive/My Drive/Colab Notebooks'

In [6]:
# Listing all the files in the directory
os.listdir(ROOT_DIR)

['ds1900.nc',
 'ds1901.nc',
 'ds1902.nc',
 'ds1904.nc',
 'supervision_notebook_01.ipynb',
 'supervision_notebook_02.ipynb']

In [8]:
# The xr.open_mfdataset function can be used to open all files in a directory
# that have '.nc'. The  wild card (*) symbol just means anything can appear
# there.
# Note os.path.join just addeds '/' between ROOT_DIR and '*.nc'
ds_joined = xr.open_mfdataset(os.path.join(ROOT_DIR, 'ds*.nc'))
ds_joined

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 77)
Coordinates:
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2013-01-20T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(20, 25, 53), meta=np.ndarray>
Attributes: (5)

In [9]:
# If you don't want all the files, then you can use 'glob'
all_netcdf_files = glob(os.path.join(ROOT_DIR, '*.nc'))
all_netcdf_files

['/content/drive/My Drive/Colab Notebooks/ds1900.nc',
 '/content/drive/My Drive/Colab Notebooks/ds1901.nc',
 '/content/drive/My Drive/Colab Notebooks/ds1902.nc',
 '/content/drive/My Drive/Colab Notebooks/ds1904.nc']

In [10]:
# Get only the last two netcdf files.
subset_of_netcdf_files = all_netcdf_files[:2]
subset_of_netcdf_files

['/content/drive/My Drive/Colab Notebooks/ds1900.nc',
 '/content/drive/My Drive/Colab Notebooks/ds1901.nc']

In [13]:
# The xr.open_mfdataset function can then be used to open all files in this list
ds_joined = xr.open_mfdataset(subset_of_netcdf_files)
ds_joined

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 39)
Coordinates:
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2013-01-10T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(20, 25, 53), meta=np.ndarray>
Attributes: (5)

# Clients

This content is a bit tricker here, but I have added it anyway. This will come in handy at some point. <p> When using large data, it is effict to split to have that computions being handled by multiple cores (essentially multiple computers). Xarray is great, as it can automatially do this for you. However, you

In [ ]:
# Don't worry about this, just creating a (somewhat) large array
shape = (2000, 100)
data = np.random.rand(365, *shape)  # Adding 365 time steps
coords = {
    'time': pd.date_range('2000-01-01', periods=365, freq='D'),  # Daily frequency for a year
    'lat': np.linspace(-90, 90, shape[0]),
    'lon': np.linspace(-180, 180, shape[1])
}
da = xr.DataArray(data, coords=coords, dims=['time', 'lat', 'lon']).chunk('auto')

In [ ]:
# This array contains dimensions time, lat, lon.
# This array has been 'chunked'. This means that the lat, lon and time
# are all split up into little section - each of these sections can be sent off
# and have a calculation perfomed on it, and then repieced.
da

<xarray.DataArray (time: 365, lat: 2000, lon: 100)>
dask.array<xarray-<this-array>, shape=(365, 2000, 100), dtype=float64, chunksize=(365, 459, 100), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-12-30
  * lat      (lat) float64 -90.0 -89.91 -89.82 -89.73 ... 89.73 89.82 89.91 90.0
  * lon      (lon) float64 -180.0 -176.4 -172.7 -169.1 ... 172.7 176.4 180.0

Chunking is a really usefull think to do to help speed of the compution of your
code. However, there are a few things you need to do to make this efficient of you may end up making your code slower.


1.   Chunks need to be kept to ~100mb
2.   The dimension you are calculting along needs to be 1 chunk (if possible).



In [ ]:
# 1 Chunks need to be kept to ~100mb

# To rechunk you can alter the chunks of your data
# E.g. below I have created chunks that are 320 lat by 200 lons by 300 in time
da.chunk({'lat':320, 'lon':200, 'time':300})

<xarray.DataArray (time: 365, lat: 2000, lon: 100)>
dask.array<rechunk-merge, shape=(365, 2000, 100), dtype=float64, chunksize=(300, 320, 100), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-12-30
  * lat      (lat) float64 -90.0 -89.91 -89.82 -89.73 ... 89.73 89.82 89.91 90.0
  * lon      (lon) float64 -180.0 -176.4 -172.7 -169.1 ... 172.7 176.4 180.0

In [ ]:
# 2.   The dimension you are calculting along needs to be 1 chunk (if possible).

# What this means, is that if you are doing an average in time, then you don't
# want time to be split up accorss one chunk - each chunk should contain all
# of time.
# E.g. in the example below shows you what that looks like

In [ ]:
# This would be a good way of chunking if you are doing an average or resmample with time
# 'time':-1 just means time will be one chunk
da.chunk({'time':-1})

<xarray.DataArray (time: 365, lat: 2000, lon: 100)>
dask.array<xarray-<this-array>, shape=(365, 2000, 100), dtype=float64, chunksize=(365, 459, 100), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-12-30
  * lat      (lat) float64 -90.0 -89.91 -89.82 -89.73 ... 89.73 89.82 89.91 90.0
  * lon      (lon) float64 -180.0 -176.4 -172.7 -169.1 ... 172.7 176.4 180.0

In [ ]:
# This would be a good way of chunking if you are doing an average in space
# 'time':-1 just means time will be one chunk
da.chunk({'time':70, 'lat':-1, 'lon':-1})

<xarray.DataArray (time: 365, lat: 2000, lon: 100)>
dask.array<rechunk-merge, shape=(365, 2000, 100), dtype=float64, chunksize=(70, 2000, 100), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-12-30
  * lat      (lat) float64 -90.0 -89.91 -89.82 -89.73 ... 89.73 89.82 89.91 90.0
  * lon      (lon) float64 -180.0 -176.4 -172.7 -169.1 ... 172.7 176.4 180.0